# 텍스트 정규화 

In [2]:
from sklearn.datasets import fetch_20newsgroups
news_data=fetch_20newsgroups(subset='all',random_state=156)
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd
print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
# 개별 데이터가 텍스트로 어떻게 구성되어있는지 확인 
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [5]:
# 내용을 제외하고 제목 등의 다른 정보는 제거하기
# 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류하기
train_news=fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),
                  random_state=156)
X_train=train_news.data
y_train=train_news.target
test_news=fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),
                  random_state=156)
X_test=test_news.data
y_test=test_news.target
print('학습 데이터의 크기 {0}, 테스트 데이터의 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터의 크기 11314, 테스트 데이터의 크기 7532


## 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 학습 데이터와 테스트 데이터 모두 리스트 형태로 주어짐
- CountVectorizer을 이용해 학습 데이터의 텍스트를 피처 벡터화함
- 이때 테스트 데이터에 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit 이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환해야 함

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cnt_vect=CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect=cnt_vect.transform(X_train)
X_test_cnt_vect=cnt_vect.transform(X_test)
print('학습 데이터의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터의 CountVectorizer Shape: (11314, 101631)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr_clf=LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
y_pred=lr_clf.predict(X_test_cnt_vect)
print('CountVectorizered Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,y_pred)))

CountVectorizered Logistic Regression의 예측 정확도는 0.607


C:\Users\MYCOM\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
# Count 기반에서 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행해보기
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect=TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)
lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
y_pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,y_pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


일반적으로 문서 내에서 택스트가 많고 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출함. 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것임. 다양한 파라미터를 적용해보기.

In [9]:
# stop words 필터링 추가하고 ngram을 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용

tfidf_vect=TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)
lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
y_pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,y_pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.692


In [10]:
# GridSearch를 이용해 로지스틱 회귀이 하이퍼 파라미터 최적화 수행
from sklearn.model_selection import GridSearchCV
params={'C':[0.01,0.1,1,5,10]}
grid_cv_lr=GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter:',grid_cv_lr.best_params_)
y_pred=grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,y_pred)))


Fitting 3 folds for each of 5 candidates, totalling 15 fits


C:\Users\MYCOM\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MYCOM\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Logistic Regression best C parameter: {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


사이킷런의 Pipeline 클래스를 이용하면 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한번에 할 수 있음. Pipeline을 통해 데이터의 전처리와 머신러닝 학습과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드를 생성할 수 있음. 또한 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있기 때문에 수행시간을 절약할 수 있음

In [11]:
from sklearn.pipeline import Pipeline
pipeline=Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),
                  ('lr_clf',LogisticRegression(C=10))])
# 별도의 객체와 함수가 필요없음
pipeline.fit(X_train,y_train)
pred=pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

C:\Users\MYCOM\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


이때 유의할 점은 모두의 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모된다는 점임

In [1]:
'''
pipeline=Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),
                  ('lr_clf'),LogisticRegression()])
params={'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
       'tfidf_vect__max_df':[100,300,700],
       'lr_clf__C':[1,5,10]}
grid_cv_pipe=GridSearchCV(pipeline,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train,y_train)
print('Logistic Regression best C parameter:',grid_cv_lr.best_param_)
y_pred=grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,y_pred)))
'''

"\npipeline=Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),\n                  ('lr_clf'),LogisticRegression()])\nparams={'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],\n       'tfidf_vect__max_df':[100,300,700],\n       'lr_clf__C':[1,5,10]}\ngrid_cv_pipe=GridSearchCV(pipeline,param_grid=params,cv=3,scoring='accuracy',verbose=1)\ngrid_cv_pipe.fit(X_train,y_train)\nprint('Logistic Regression best C parameter:',grid_cv_lr.best_param_)\ny_pred=grid_cv_pipe.predict(X_test)\nprint('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,y_pred)))\n"